## Deployments

- We have seen how to use standalone K8s pods
- Now we'll see how to scale with K8s using deployments

- Note that all pods within a single deployment will be exactly the same

- `kubectl create deployment nginx-deployment --image=nginx`
    - Create deployment
- `kubectl get deployments` / `kubectl get pods`
    - Get deployment and pod details
- `kubectl describe deployment nginx-deployment`
    - Get details of created deployment
    - You will see a **ReplicaSet** tagged to a deployment. What is this?
        - ReplicaSet manages all pods related to deployment; simply a set of replicas of your application

- `kubectl get pods`
    - Notice that the name of the pod now starts with the name of the replicaset, followed by a specific hash for each pod

- `kubectl describe pod nginx-deployment-c45d79c8-cmfws`
    - Get single pod details

- `kubectl scale deployment nginx-deployment --replicas=5`
    - Create 4 new containers that duplicates your ealrier deployment

- `kubectl get pods -o wide`
    - See that each of your pods have a different IP address

- `kubectl scale deployment nginx-deployment --replicas=3`
    - Scale down to 3 replicas instead

- `minikube ssh` + `curl 10.244.0.6`
    - ssh into your kubernetes cluster, and curl the pod

- So far, we've tried to connect to these pods via their IP addresses
    - This is a terrible way of doing it, because obviously the allocated address changes every time
    - In K8s cluster, it is possible to have multiple nodes, and pods can be distributed inside different nodes
        - So trying to access using IP address, or even virtual IP addresses, is going to be a massive pain, because often IP addresses are private to the node
        - So instead, the easier way is to have a load balancer IP address
        - In this way, there is only 1 IP address for the whole K8s cluster for a deployment, and you can access your deployment with a single entry point
        - Such load balancer IP address is usually assigned by specific cloud provider, which is controlled by `cloud controller manager` which runs on master node

- `kubectl get deployments`

- `kubectl expose deployment nginx-deployment --port=8080 --target-port=80`
    - We create a service for this deployment
    - Using this, we can expose a specific pod from the deployment 
    - by default, nginx runs on port 80, so we expose internal port 80 on the containers to any other pods outside of the deployment

- `kubectl get services`
    - Now, there are 2 services; one for the main K8s deployment, and the other for the nginx-deployment we just exposed
    - Notice that the cluster IP address for the exposed deployment exists in another network, and can now be used to connect to any pod from inside the K8s cluster
    - Note that this cluster IP address is accessible from within your K8s deployment (i.e. after you ssh), but not from the outside world

- `minikube ssh` + `curl 10.105.236.114:8080`
    - The curl will hit one of the pods in the deployment, but you won't know which exactly
    - K8s will handle all load balancing to specific pods

- `kubectl describe service nginx-deployment`